# Keras based notebook for QA

The code used in this notebook is largely derived from two excellent sources:
https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
https://keras.io/examples/nlp/text_extraction_with_bert/

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import re
import pandas as pd
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers.modeling_tf_utils import shape_list
from transformers import BertTokenizerFast, TFBertModel,TFXLMRobertaModel,XLMRobertaTokenizerFast
from sklearn.model_selection import train_test_split

In [ ]:
max_length = 384
doc_stride = 128

In [ ]:
#tokenizer = BertTokenizerFast.from_pretrained("../input/bert-base-multilingual-cased/bert-base-multilingual-cased")
tokenizer = XLMRobertaTokenizerFast.from_pretrained("../input/jplu-tf-xlm-roberta-large")

In [ ]:
train = pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
print(test.shape)
test.head()

In [ ]:
train,val = train_test_split(train,test_size=0.20,random_state=2021)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        list(examples["question"].values),
        list(examples["context"].values),
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")
    
    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples.loc[sample_index,"answer_text"]
        answer_start = examples.loc[sample_index,"answer_start"]
        
        # If no answers are given, set the cls_index as answer.
        if answer_start is None:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answer_start
            end_char = start_char + len(answers)

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        list(examples["question"].values),
        list(examples["context"].values),
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples.loc[sample_index,"id"])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
train_encodings  =  prepare_train_features(train)
val_encodings    =  prepare_validation_features(val)
test_encodings   =  prepare_validation_features(test)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key] for key in ['start_positions', 'end_positions']}
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val_encodings[key] for key in ['input_ids', 'attention_mask']},
    
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    {key: test_encodings[key] for key in ['input_ids', 'attention_mask']},
    
))

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))
val_dataset = val_dataset.map(lambda x: x)
test_dataset = test_dataset.map(lambda x: x)

In [ ]:
def create_model():
    ## encoder
    #encoder = TFBertModel.from_pretrained("../input/bert-base-multilingual-cased/bert-base-multilingual-cased")
    encoder = TFXLMRobertaModel.from_pretrained("../input/jplu-tf-xlm-roberta-large")
    ## QA Model
    input_ids = layers.Input(shape=(max_length,), dtype=tf.int32,name='input_ids')
    #token_type_ids = layers.Input(shape=(max_length,), dtype=tf.int32,name='token_type_ids')
    attention_mask = layers.Input(shape=(max_length,), dtype=tf.int32,name='attention_mask')
    embedding = encoder(
        input_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax,name='start_positions')(start_logits)
    end_probs = layers.Activation(keras.activations.softmax,name='end_positions')(end_logits)

    model = keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(8), epochs=1, batch_size=8)

In [ ]:
start,end = model.predict(val_dataset.batch(4))

In [ ]:
start.shape,end.shape

In [ ]:
from tqdm.auto import tqdm
import collections

def postprocess_qa_predictions(examples, features, start,end, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = start,end
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features["example_id"]):
        
        features_per_example[example_id_to_index[feature]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features['input_ids'])} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples.context.values)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features["offset_mapping"][feature_index]

            # Update minimum null prediction.
            cls_index = features["input_ids"][feature_index].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        #if not squad_v2:
        #    predictions[example["id"]] = best_answer["text"]
        #else:
        answer = best_answer["text"] 
        predictions[example_index] = answer

    return predictions

In [ ]:
val_predictions = postprocess_qa_predictions(val, val_encodings, start,end)

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
score = []
for idx in range(len(val)):
    str1 = val.answer_text.values[idx]
    str2 = val_predictions[idx]
    score.append(jaccard(str1,str2))

In [ ]:
np.array(score).mean()

In [ ]:
sub = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
sub.head()

In [ ]:
start,end = model.predict(test_dataset.batch(4))
start.shape,end.shape

In [ ]:
test_predictions = postprocess_qa_predictions(test, test_encodings, start,end)

In [ ]:
test_preds = []
for idx in range(len(test)):
    test_preds.append(test_predictions[idx])


In [ ]:
test['PredictionString'] = test_preds
test[['id','PredictionString']].head()

In [ ]:
test[['id','PredictionString']].to_csv('submission.csv',index=False)